In [32]:
from bs4 import BeautifulSoup

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [34]:
# 1️⃣ 크롬 드라이버 실행
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(options=options)

In [35]:
# 2️⃣ 쇼츠 URL 열기
url = "https://www.youtube.com/shorts/PFiyCYfNn80"
driver.get(url)

In [36]:
time.sleep(5)  # 페이지 초기 로딩 대기

In [23]:
comments_button_container = driver.find_element(By.ID, "comments-button")

In [24]:
button = comments_button_container.find_element(By.TAG_NAME, "button")
button.click()

In [25]:
threads = driver.find_elements(By.TAG_NAME, 'ytd-comment-thread-renderer')

In [26]:
seen_comments = set()

In [27]:
# new_found = 0
    
# for thread in threads:
#     try:
#         author = thread.find_element(By.ID, 'author-text').text
#         content = thread.find_element(By.ID, 'content-text').text
#         unique_key = (author, content)

#         if unique_key not in seen_comments:
#             seen_comments.add(unique_key)
#             print(f"Author: {author}")
#             print(f"Content: {content}")
#             new_found += 1

#     except Exception as e:
#         print("Error:", e)

In [28]:
scroll_candidates = [
    "#contents",
    "ytd-item-section-renderer",
    "ytd-comments",
    "ytd-section-list-renderer",
    "ytd-engagement-panel-section-list-renderer"
]


In [29]:
for sel in scroll_candidates:
    try:
        el = driver.find_element(By.CSS_SELECTOR, sel)
        # overflow-y 속성 확인
        overflow = driver.execute_script(
            "return window.getComputedStyle(arguments[0]).overflowY;", el
        )
        print(f"{sel}: overflow-y = {overflow}")

        # 스크롤 시도
        driver.execute_script(
            "arguments[0].scrollTop += 500;", el
        )
        print(f"Scrolled {sel}")

        time.sleep(2)
    except Exception as e:
        print(f"Failed for {sel}: {e}")


#contents: overflow-y = visible
Scrolled #contents
ytd-item-section-renderer: overflow-y = visible
Scrolled ytd-item-section-renderer
ytd-comments: overflow-y = visible
Scrolled ytd-comments
ytd-section-list-renderer: overflow-y = hidden
Scrolled ytd-section-list-renderer
ytd-engagement-panel-section-list-renderer: overflow-y = hidden
Scrolled ytd-engagement-panel-section-list-renderer


In [37]:
import time
from selenium.webdriver.common.by import By

# 1️⃣ 댓글 버튼 열기
try:
    comment_button = driver.find_element(By.CSS_SELECTOR, "#comments-button")
    comment_button.click()
    print("✅ 댓글 버튼 클릭 완료")
    time.sleep(3)
except Exception as e:
    print("❌ 댓글 버튼 못 찾음:", e)


# 2️⃣ 댓글창 스크롤 컨테이너 찾기
# 가장 많이 성공 사례에서 쓰던 패턴
try:
    scroll_container = driver.find_element(
        By.CSS_SELECTOR,
        "ytd-item-section-renderer #contents"
    )
    print("✅ 스크롤 대상 찾음")
except Exception as e:
    print("❌ 스크롤 대상 못 찾음:", e)
    scroll_container = None

time.sleep(2)


# 3️⃣ 스크롤 루프
if scroll_container:
    last_height = driver.execute_script("return arguments[0].scrollHeight;", scroll_container)

    for i in range(20):
        print(f"👉 스크롤 {i+1}회")
        driver.execute_script(
            "arguments[0].scrollTop = arguments[0].scrollHeight;", 
            scroll_container
        )
        time.sleep(2)

        new_height = driver.execute_script("return arguments[0].scrollHeight;", scroll_container)
        print(f"📏 이전 높이: {last_height} -> 현재 높이: {new_height}")

        if new_height == last_height:
            print("✅ 더 이상 로딩할 댓글 없음. 스크롤 종료.")
            break

        last_height = new_height

    print("✅ 스크롤 루프 종료")
else:
    print("❌ 스크롤 컨테이너가 없어서 스크롤 못 돌림")


✅ 댓글 버튼 클릭 완료
✅ 스크롤 대상 찾음
👉 스크롤 1회
📏 이전 높이: 3236 -> 현재 높이: 5772
👉 스크롤 2회
📏 이전 높이: 5772 -> 현재 높이: 8232
👉 스크롤 3회
📏 이전 높이: 8232 -> 현재 높이: 10688
👉 스크롤 4회


KeyboardInterrupt: 